## Ryan VacationPy starts here



In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import geoviews as gv
import geoviews.tile_sources as gvts
import matplotlib.pyplot as plt
import matplotlib as mplb
import numpy as np
import time
import random
import scipy.stats as st
from geoviews import dim, opts
from api_keys import geoapifykey 

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,huinan,42.6225,126.2614,19.08,48,9,4.49,CN,1685752646
1,1,tuktoyaktuk,69.4541,-133.0374,10.00,62,75,6.17,CA,1685752647
2,2,albany,42.6001,-73.9662,20.53,96,81,0.76,US,1685752381
3,3,atuona,-9.8000,-139.0333,26.80,72,1,4.44,PF,1685752650
4,4,narsaq,60.9167,-46.0500,7.12,99,100,1.43,GL,1685752651


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# configure map plot

map_plot_h = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=0.8,
    color="City",
    colorbar=True,
    hover_cols=["City", "Lat","Lng","Humidity"]
)

# display map plot
map_plot_h

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
select_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 29) &
    (city_data_df["Max Temp"] > 23) & 
    (city_data_df["Wind Speed"] < 2.5) &
    (city_data_df["Cloudiness"] < 10) &
    (city_data_df["Humidity"] < 80 )
]

# Drop any rows with null values
select_city_df = select_city_df.dropna()

# Display sample data
select_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
132,132,fredericksburg,38.3032,-77.4605,27.12,68,0,0.00,US,1685752755
186,186,rolla,37.9514,-91.7713,23.04,78,0,2.06,US,1685752741
353,353,safita,34.8205,36.1153,23.72,43,7,0.29,SY,1685752996
430,430,russell,32.3502,-85.1999,28.78,74,3,1.68,US,1685753070
541,541,baherden,38.4362,57.4316,26.33,42,0,1.54,TM,1685753180


### Step 3: Create a new DataFrame called `hotel_df`

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = select_city_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
132,132,fredericksburg,38.3032,-77.4605,27.12,68,0,0.00,US,1685752755,
186,186,rolla,37.9514,-91.7713,23.04,78,0,2.06,US,1685752741,
353,353,safita,34.8205,36.1153,23.72,43,7,0.29,SY,1685752996,
430,430,russell,32.3502,-85.1999,28.78,74,3,1.68,US,1685753070,
541,541,baherden,38.4362,57.4316,26.33,42,0,1.54,TM,1685753180,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
category = "accommodation.hotel"


params = {
    "categories":category,
    "filter":"circle:",
    "bias" : "proximity:",
    "sort":"distance",
    "limit":1,
    "apiKey": geoapifykey

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fredericksburg - nearest hotel: Kenmore Inn
rolla - nearest hotel: Pear Tree Inn Rolla
safita - nearest hotel: فندق روز ماري
russell - nearest hotel: No hotel found
baherden - nearest hotel: No hotel found
quesnel - nearest hotel: Quality Inn
houma - nearest hotel: Courtyard by Marriott Houma
mohan - nearest hotel: No hotel found
satitoa - nearest hotel: No hotel found
bom jesus - nearest hotel: Hotel Brasão
armacao dos buzios - nearest hotel: Hotel Brasão


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
132,132,fredericksburg,38.3032,-77.4605,27.12,68,0,0.00,US,1685752755,Kenmore Inn
186,186,rolla,37.9514,-91.7713,23.04,78,0,2.06,US,1685752741,Pear Tree Inn Rolla
353,353,safita,34.8205,36.1153,23.72,43,7,0.29,SY,1685752996,فندق روز ماري
430,430,russell,32.3502,-85.1999,28.78,74,3,1.68,US,1685753070,No hotel found
541,541,baherden,38.4362,57.4316,26.33,42,0,1.54,TM,1685753180,No hotel found
676,676,quesnel,52.9999,-122.5029,23.28,35,0,2.06,CA,1685753110,Quality Inn
688,688,houma,29.5958,-90.7195,28.89,62,0,2.06,US,1685753328,Courtyard by Marriott Houma
710,710,mohan,26.7833,80.6833,28.97,33,0,2.28,IN,1685753371,No hotel found
711,711,satitoa,26.7833,80.6833,28.97,33,0,2.28,IN,1685753371,No hotel found
716,716,bom jesus,-9.0744,-44.3586,24.32,49,7,0.99,BR,1685753377,Hotel Brasão


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.


In [15]:
# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=0.8,
    color="City",
    colorbar=True,
    hover_cols=["Lat","Lng","Humidity","City", "Hotel Name","Country" ]
)

# display map plot
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)